In [1]:
from glob import glob
import pandas as pd
import numpy as np
import re
import cv2
from tqdm import tqdm

In [2]:
outputs = glob('outputs/*')
# frames = glob('cam1/*')
frame_nums = []

for i in outputs:
    frame_nums.append(re.findall("\d+", i)[0])
    
table = pd.DataFrame(frame_nums, columns=['frame_num'])
table['output'] = pd.Series(outputs)
table['raw'] = 'cam1/' + table['frame_num'] + '.jpg'
table['frame_num'] = table['frame_num'].astype(int)
table['homo'] = table['raw'].str.replace('cam1', 'images')
table['label'] = table['raw'].str.replace('cam1', 'labels')

table.sort_values('frame_num').reset_index(drop=True)

table.sort_values('frame_num', inplace=True)
table.reset_index(drop=True, inplace=True)

pd.to_pickle(table, 'table.pkl')

In [3]:
pts_src1 = np.array([[614, 294], [581, 326],[503, 407], [313, 348], [478, 345]])
pts_dst = np.array([[650, 550], [600, 550], [500, 550],[500, 400], [550, 500]]) #these are the zoomed out destination points
homo = cv2.findHomography(pts_src1, pts_dst)[0]

y = np.empty((len(table), 17, 2))

for i, row in tqdm(table.iterrows(), total=len(table)):
    undis = cv2.imread(row.raw)
    transformed = cv2.warpPerspective(undis, homo, (undis.shape[1], undis.shape[0]))
    cv2.imwrite(row.homo, transformed)
    
    points = pd.read_csv(table.iloc[0].output).values[:, 1:]
    points = cv2.perspectiveTransform(points.reshape(-1, 1, 2).astype(np.float32), homo)
    points /= transformed.shape[:2] #scale to the dimensions of the image
    y[i] = points.reshape(17, 2)
    
    with open(row.label, 'w') as f:
        for xcoord, ycoord in y[i]:
            f.write(f'0 {xcoord} {ycoord} 0 0')
    
np.save('y.npy', y)

100%|██████████| 1633/1633 [00:34<00:00, 46.84it/s]


In [6]:
a = table.sample(frac=1)
a

,frame_num,output,raw,homo,label
1602,28400,outputs/output00028400.csv,cam1/00028400.jpg,images/00028400.jpg,labels/00028400.jpg
8,49,outputs/output00000049.csv,cam1/00000049.jpg,images/00000049.jpg,labels/00000049.jpg
711,10580,outputs/output00010580.csv,cam1/00010580.jpg,images/00010580.jpg,labels/00010580.jpg
788,12120,outputs/output00012120.csv,cam1/00012120.jpg,images/00012120.jpg,labels/00012120.jpg
1529,26940,outputs/output00026940.csv,cam1/00026940.jpg,images/00026940.jpg,labels/00026940.jpg
...,...,...,...,...,...
649,9340,outputs/output00009340.csv,cam1/00009340.jpg,images/00009340.jpg,labels/00009340.jpg
244,1240,outputs/output00001240.csv,cam1/00001240.jpg,images/00001240.jpg,labels/00001240.jpg
1482,26000,outputs/output00026000.csv,cam1/00026000.jpg,images/00026000.jpg,labels/00026000.jpg
1632,29000,outputs/output00029000.csv,cam1/00029000.jpg,images/00029000.jpg,labels/00029000.jpg


In [19]:
type(a.iloc[0]._name)

numpy.int64